#SNA2 : Social Network Analysis Project

This is "[a] network of books about US politics published around the time of the 2004 presidential election and sold by the online bookseller Amazon.com. Edges between books represent frequent copurchasing of books by the same buyers. The network was compiled by V. Krebs and is unpublished, but can found on Krebs' web site (http://www.orgnet.com/).


##**1. Data Preparation**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

# change path to the designated google drive folder
# otherwise, data will be saved in /content folder which you may have issue locating
%cd /content/drive/Shared drives/SNA group project

Mounted at /content/drive
/content/drive/Shared drives/SNA group project


In [2]:
import networkx as nx
!pip install ipysigma
from ipysigma import Sigma
import seaborn as sns


poltext = nx.read_gml('polbooks.gml', label='label')


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 69.9 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np

#Visualization libraries
import plotly.express as px
from matplotlib import pyplot as plt

##**2. Exploratory Data Analysis**

In [4]:
df = pd.read_csv('polbooks.csv') 
df

,Id,Label,Interval,Value
0,0,1000 Years for Revenge,NaN,n
1,1,Bush vs. the Beltway,NaN,c
2,2,Charlie Wilson's War,NaN,c
3,3,Losing Bin Laden,NaN,c
4,4,Sleeping With the Devil,NaN,n
...,...,...,...,...
100,100,Thieves in High Places,NaN,l
101,101,Shrub,NaN,l
102,102,Buck Up Suck Up,NaN,l
103,103,The Future of Freedom,NaN,n


In [5]:
import plotly.express as px


labels = {'c': 'Conservative', 'n': 'Neutral', 'l': 'Liberal'}

df = df.replace(labels)
# create a histogram using Plotly
fig = px.histogram(df, x='Value', color='Value',
                   color_discrete_map={'Conservative': '#D25565', 'Neutral': '#F0B775', 'Liberal': '#2E94B9'})

# update the layout of the figure
fig.update_layout(title='Distribution of Political Books on Dataset',
                  xaxis_title='Political Orientation',
                  yaxis_title='Count',
                  xaxis={'categoryorder': 'total descending'},
                  height=400, width=500)

# show the plot
fig.show()

## **3. Social Network Analysis**

In [6]:
from google.colab import output
output.enable_custom_widget_manager()

In [7]:
mapping = {'c': 'Conservation', 'l': 'Liberal', 'n': 'Neutral'}

# update the 'value' attribute with new labels
for node, attr in poltext.nodes(data=True):
    old_label = attr['value']
    new_label = mapping.get(old_label, old_label)
    attr['value'] = new_label

In [8]:
color_map = {'Conservation': '#D25565', 'Neutral': '#F0B775', 'Liberal': '#2E94B9'}


Sigma(poltext,node_label='label',node_size=poltext.degree(), node_color="value", node_border_color_from="node", node_color_palette=color_map)

Sigma(nx.Graph with 105 nodes and 441 edges)

In [9]:
Sigma(poltext, node_metrics=["louvain"], node_color="louvain")

# Renaming the target attribute
Sigma(poltext, node_metrics={"community": "louvain"}, node_color="community")

# Passing custom parameters
Sigma(
  poltext,
  node_metrics={"community": {"name": "louvain", "resolution": 1}},
  node_color="community", node_size=poltext.degree(), node_border_color_from="node"
)

Sigma(nx.Graph with 105 nodes and 441 edges)

### 3.1      Degree Centrality

This metric measures the number of connections (or edges) each node (or book) has. In the context of this network, books with high degree centrality would be those that were frequently copurchased with many other books.

https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.centrality.degree_centrality.html

In [10]:
#Calculate Degree Centrality
degree_centrality = nx.degree_centrality(poltext)

top_books = sorted(degree_centrality.items(), key=lambda x: x[1], reverse=True)[:20]
for book, centrality in top_books:
    print(f'{book}: {centrality}')

A National Party No More: 0.2403846153846154
Off with Their Heads: 0.2403846153846154
Losing Bin Laden: 0.22115384615384617
Bushwhacked: 0.22115384615384617
American Dynasty: 0.21153846153846156
The Great Unraveling: 0.20192307692307693
Big Lies: 0.20192307692307693
The Price of Loyalty: 0.19230769230769232
Legacy: 0.17307692307692307
Deliver Us from Evil: 0.17307692307692307
Arrogance: 0.17307692307692307
Bush Country: 0.15384615384615385
The Lies of George W. Bush: 0.15384615384615385
Worse Than Watergate: 0.15384615384615385
Dereliction of Duty: 0.14423076923076925
Against All Enemies: 0.14423076923076925
Lies and the Lying Liars Who Tell Them: 0.13461538461538464
Persecution: 0.125
Rise of the Vulcans: 0.125
Plan of Attack: 0.125


In [11]:
# add a new column to the DataFrame with the prices from the dictionary
df['degree_centrality'] = df['Label'].map(degree_centrality)

In [12]:
# sort the DataFrame by degree centrality in descending order and select the top 10 rows
top_3_degree_centrality = df.sort_values(by='degree_centrality', ascending=False).head(3)

# count the number of points in each category within the top 10
category_counts = top_3_degree_centrality['Value'].value_counts()

# print the category counts
print(category_counts)

Conservative    3
Name: Value, dtype: int64


In [13]:
fig = px.histogram(top_3_degree_centrality, x='Value', color='Value',
                   color_discrete_map={'Conservative': '#D25565', 'Neutral': '#F0B775', 'Liberal': '#2E94B9'})

# update the layout of the figure
fig.update_layout(title='Top 3 Political Books on Amazon with Highest Degree Centrality',
                  xaxis_title='Political Orientation',
                  yaxis_title='Count',
                  xaxis={'categoryorder': 'total descending'},
                  height=400, width=700)

# show the plot
fig.show()

In [14]:
N_mis=len(poltext.nodes())
E_mis=len(poltext.edges())
N_mis,E_mis,E_mis/N_mis

(105, 441, 4.2)

In [15]:
from matplotlib import pyplot as plt
from networkx.readwrite import gml
G = nx.watts_strogatz_graph(n = N_mis, k = 3, p = 0.05)
pos = nx.circular_layout(G)
plt.figure(figsize = (12, 12))
Sigma(G, node_label='label')

Sigma(nx.Graph with 105 nodes and 105 edges)

<Figure size 1200x1200 with 0 Axes>

### 3.2      betweenness_centrality

This metric measures the extent to which a node lies on the shortest paths between other nodes in the network. In the context of this network, books with high betweenness centrality would be those that were frequently copurchased with other books that were not necessarily directly connected to each other.

https://networkx.org/documentation/stable/auto_examples/algorithms/plot_betweenness_centrality.html

In [16]:
#Calculate Betweeness Centrality
betweenness_centrality = nx.betweenness_centrality(poltext)

top_books_betweeness = sorted(betweenness_centrality.items(), key=lambda x: x[1], reverse=True)[:10]
for book, centrality in top_books:
    print(f'{book}: {centrality}')

A National Party No More: 0.2403846153846154
Off with Their Heads: 0.2403846153846154
Losing Bin Laden: 0.22115384615384617
Bushwhacked: 0.22115384615384617
American Dynasty: 0.21153846153846156
The Great Unraveling: 0.20192307692307693
Big Lies: 0.20192307692307693
The Price of Loyalty: 0.19230769230769232
Legacy: 0.17307692307692307
Deliver Us from Evil: 0.17307692307692307
Arrogance: 0.17307692307692307
Bush Country: 0.15384615384615385
The Lies of George W. Bush: 0.15384615384615385
Worse Than Watergate: 0.15384615384615385
Dereliction of Duty: 0.14423076923076925
Against All Enemies: 0.14423076923076925
Lies and the Lying Liars Who Tell Them: 0.13461538461538464
Persecution: 0.125
Rise of the Vulcans: 0.125
Plan of Attack: 0.125


In [17]:
betweenness_centrality

{'1000 Years for Revenge': 0.007433088834022092,
 'Bush vs. the Beltway': 4.890898464136117e-05,
 "Charlie Wilson's War": 0.0005207326902621901,
 'Losing Bin Laden': 0.07609290805120641,
 'Sleeping With the Devil': 0.06292846885423584,
 'The Man Who Warned America': 0.012880661322399524,
 'Why America Slept': 0.0562973388998594,
 'Ghost Wars': 0.06872446488084949,
 'A National Party No More': 0.07490760424804607,
 'Bush Country': 0.09839490722763909,
 'Dereliction of Duty': 0.0337318994217265,
 'Legacy': 0.02928266458085173,
 'Off with Their Heads': 0.09526168061799924,
 'Persecution': 0.01802632594468501,
 "Rumsfeld's War": 0.05884272084382055,
 'Breakdown': 0.0007686618337207065,
 'Betrayal': 0.0,
 'Shut Up and Sing': 0.00012231472726953143,
 'Meant To Be': 4.890898464136117e-05,
 'The Right Man': 0.036539926798552794,
 'Ten Minutes from Normal': 0.02622857624427163,
 "Hillary's Scheme": 0.00012432955394120444,
 'The French Betrayal of America': 0.007157357840135858,
 'Tales from the

In [18]:
# add a new column to the DataFrame with the prices from the dictionary
df['betweenness_centrality'] = df['Label'].map(betweenness_centrality)

In [19]:
# sort the DataFrame by degree centrality in descending order and select the top 10 rows
top_20_centrality = df.sort_values(by='betweenness_centrality', ascending=False).head(3)

# count the number of points in each category within the top 10
category_counts = top_20_centrality['Value'].value_counts()

# print the category counts
print(category_counts)
print(top_20_centrality)

Conservative    2
Liberal         1
Name: Value, dtype: int64
    Id                 Label  Interval         Value  degree_centrality  \
30  30  The Price of Loyalty       NaN       Liberal           0.192308   
49  49            The Bushes       NaN  Conservative           0.076923   
9    9          Bush Country       NaN  Conservative           0.153846   

    betweenness_centrality  
30                0.139478  
49                0.103649  
9                 0.098395  


In [20]:
fig = px.histogram(top_20_centrality, x='Value', color='Value',
                   color_discrete_map={'Conservative': '#D25565', 'Neutral': '#F0B775', 'Liberal': '#2E94B9'})

# update the layout of the figure
fig.update_layout(title='Top 20 Political Books on Amazon with Highest Betweenness Centrality',
                  xaxis_title='Political Orientation',
                  yaxis_title='Count',
                  xaxis={'categoryorder': 'total descending'},
                  height=400, width=700)

# show the plot
fig.show()

### 3.3      Community Detection (Modularity Maximisation)

This analysis involves identifying groups of nodes that are more tightly connected to each other than to the rest of the network. In the context of this network, communities could represent clusters of books that were frequently copurchased together, perhaps corresponding to different political ideologies or topics. 

This section use Modularity Maximisation Algorithm

https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.community.modularity_max.greedy_modularity_communities.html#networkx.algorithms.community.modularity_max.greedy_modularity_communities


In [21]:
import community

community_greedy_modularity = nx.community.greedy_modularity_communities(poltext, weight=None, resolution=1, cutoff=1, best_n=None)
community_greedy_modularity

[frozenset({'1000 Years for Revenge',
            'A National Party No More',
            'Arrogance',
            'Betrayal',
            'Bias',
            'Breakdown',
            'Bush Country',
            'Bush vs. the Beltway',
            "Charlie Wilson's War",
            'Dangerous Dimplomacy',
            'Deliver Us from Evil',
            'Dereliction of Duty',
            'Endgame',
            'Fighting Back',
            'Ghost Wars',
            'Give Me a Break',
            'Hating America',
            "Hillary's Scheme",
            'Hollywood Interrupted',
            'Legacy',
            'Let Freedom Ring',
            'Losing Bin Laden',
            'Meant To Be',
            'Off with Their Heads',
            'Persecution',
            'Power Plays',
            "Rumsfeld's War",
            'Shut Up and Sing',
            'Slander',
            'Sleeping With the Devil',
            'Spin Sisters',
            'Tales from the Left Coast',
            'Ten 

In [22]:
community_list =[]
for i in community_greedy_modularity:
  list_item = list(i)
  community_list.append(list_item)
community_list

[["Who's Looking Out for You?",
  'Slander',
  'The Right Man',
  'Shut Up and Sing',
  'Those Who Trespass',
  'Dangerous Dimplomacy',
  'Hating America',
  'The French Betrayal of America',
  'Losing Bin Laden',
  'The Death of Right and Wrong',
  'The Third Terrorist',
  "Rumsfeld's War",
  'Betrayal',
  'Bush Country',
  'Tales from the Left Coast',
  'Fighting Back',
  'The Enemy Within',
  'Breakdown',
  'We Will Prevail',
  'Power Plays',
  'Give Me a Break',
  'A National Party No More',
  '1000 Years for Revenge',
  'Ghost Wars',
  'Useful Idiots',
  "Hillary's Scheme",
  'Why Courage Matters',
  'Dereliction of Duty',
  'The Real America',
  'Ten Minutes from Normal',
  'Off with Their Heads',
  'Legacy',
  'Hollywood Interrupted',
  'Bush vs. the Beltway',
  'Deliver Us from Evil',
  'Let Freedom Ring',
  'Persecution',
  'The Man Who Warned America',
  'Spin Sisters',
  "Charlie Wilson's War",
  "The O'Reilly Factor",
  'Endgame',
  'Arrogance',
  'The Savage Nation',
  'Bi

In [23]:
df_community = pd.DataFrame
community_1 = community_list[0]
community_2 = community_list[1]
community_3 = community_list[2]
community_4 = community_list[3]

book_list = []
for i in community_1:
  book_list.append(i)
for i in community_2:
  book_list.append(i)
for i in community_3:
  book_list.append(i)
for i in community_4:
  book_list.append(i)

print(len(book_list))

community_dict = {}

for item in book_list:
        if item in community_1:
            community_dict[item] = 1
        if item in community_2:
            community_dict[item] = 2
        if item in community_3:
            community_dict[item] = 3
        if item in community_4:
            community_dict[item] = 4



print(community_dict)

105
{"Who's Looking Out for You?": 1, 'Slander': 1, 'The Right Man': 1, 'Shut Up and Sing': 1, 'Those Who Trespass': 1, 'Dangerous Dimplomacy': 1, 'Hating America': 1, 'The French Betrayal of America': 1, 'Losing Bin Laden': 1, 'The Death of Right and Wrong': 1, 'The Third Terrorist': 1, "Rumsfeld's War": 1, 'Betrayal': 1, 'Bush Country': 1, 'Tales from the Left Coast': 1, 'Fighting Back': 1, 'The Enemy Within': 1, 'Breakdown': 1, 'We Will Prevail': 1, 'Power Plays': 1, 'Give Me a Break': 1, 'A National Party No More': 1, '1000 Years for Revenge': 1, 'Ghost Wars': 1, 'Useful Idiots': 1, "Hillary's Scheme": 1, 'Why Courage Matters': 1, 'Dereliction of Duty': 1, 'The Real America': 1, 'Ten Minutes from Normal': 1, 'Off with Their Heads': 1, 'Legacy': 1, 'Hollywood Interrupted': 1, 'Bush vs. the Beltway': 1, 'Deliver Us from Evil': 1, 'Let Freedom Ring': 1, 'Persecution': 1, 'The Man Who Warned America': 1, 'Spin Sisters': 1, "Charlie Wilson's War": 1, "The O'Reilly Factor": 1, 'Endgame':

In [24]:
df['communtiy_modularity'] = df['Label'].map(community_dict)

In [25]:
df

,Id,Label,Interval,Value,degree_centrality,betweenness_centrality,communtiy_modularity
0,0,1000 Years for Revenge,NaN,Neutral,0.057692,0.007433,1
1,1,Bush vs. the Beltway,NaN,Conservative,0.038462,0.000049,1
2,2,Charlie Wilson's War,NaN,Conservative,0.038462,0.000521,1
3,3,Losing Bin Laden,NaN,Conservative,0.221154,0.076093,1
4,4,Sleeping With the Devil,NaN,Neutral,0.076923,0.062928,1
...,...,...,...,...,...,...,...
100,100,Thieves in High Places,NaN,Liberal,0.115385,0.007119,2
101,101,Shrub,NaN,Liberal,0.048077,0.000943,2
102,102,Buck Up Suck Up,NaN,Liberal,0.038462,0.040995,2
103,103,The Future of Freedom,NaN,Neutral,0.019231,0.000000,3


In [26]:
category_counts = df['communtiy_modularity'].value_counts()

# print the category counts
print(category_counts)

1    49
2    41
3    12
4     3
Name: communtiy_modularity, dtype: int64


In [27]:
fig = px.histogram(df, x='communtiy_modularity', color='Value', barmode='group',
            color_discrete_map={'Conservative': '#D25565', 'Neutral': '#F0B775', 'Liberal': '#2E94B9'})

# update the layout of the figure
fig.update_layout(title='Community Detection of Political Books on Amazon with Modularity Maximisation',
                  xaxis_title='Community',
                  yaxis_title='Count',
                  xaxis={'categoryorder': 'total descending'},
                  height=400, width=900)

# show the plot
fig.show()

### 3.4      Community Detection (Louvain Algorithm)

This analysis involves identifying groups of nodes that are more tightly connected to each other than to the rest of the network. In the context of this network, communities could represent clusters of books that were frequently copurchased together, perhaps corresponding to different political ideologies or topics. 

This section use Louvain Algorithm 

https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.community.louvain.louvain_communities.html#networkx.algorithms.community.louvain.louvain_communities


In [35]:
import community

community_louvain= nx.community.louvain_communities(poltext, weight=None, resolution=1, threshold=1e-07, seed=None)

community_louvain

[{'1000 Years for Revenge',
  'Bush vs. the Beltway',
  "Charlie Wilson's War",
  'Dangerous Dimplomacy',
  'Ghost Wars',
  'Sleeping With the Devil',
  'The Man Who Warned America',
  'Why America Slept'},
 {'A National Party No More',
  'Arrogance',
  'Betrayal',
  'Bias',
  'Breakdown',
  'Bush Country',
  'Deliver Us from Evil',
  'Dereliction of Duty',
  'Endgame',
  'Fighting Back',
  'Give Me a Break',
  'Hating America',
  "Hillary's Scheme",
  'Hollywood Interrupted',
  'Legacy',
  'Let Freedom Ring',
  'Losing Bin Laden',
  'Meant To Be',
  'Off with Their Heads',
  'Persecution',
  'Power Plays',
  "Rumsfeld's War",
  'Shut Up and Sing',
  'Slander',
  'Spin Sisters',
  'Tales from the Left Coast',
  'Ten Minutes from Normal',
  'The Death of Right and Wrong',
  'The Enemy Within',
  'The French Betrayal of America',
  "The O'Reilly Factor",
  'The Official Handbook Vast Right Wing Conspiracy',
  'The Real America',
  'The Right Man',
  'The Savage Nation',
  'The Third Terr

In [36]:
community_list =[]
for i in community_louvain:
  list_item = list(i)
  community_list.append(list_item)
community_list

[["Charlie Wilson's War",
  'Ghost Wars',
  'Sleeping With the Devil',
  'Why America Slept',
  'Dangerous Dimplomacy',
  'Bush vs. the Beltway',
  '1000 Years for Revenge',
  'The Man Who Warned America'],
 ["Who's Looking Out for You?",
  'Slander',
  'The Right Man',
  'Shut Up and Sing',
  'Those Who Trespass',
  'The French Betrayal of America',
  'Hating America',
  'Losing Bin Laden',
  'The Death of Right and Wrong',
  'The Third Terrorist',
  "Rumsfeld's War",
  'Betrayal',
  'Bush Country',
  'Fighting Back',
  'Tales from the Left Coast',
  'The Enemy Within',
  'Breakdown',
  'We Will Prevail',
  'Power Plays',
  'Give Me a Break',
  'A National Party No More',
  'Useful Idiots',
  "Hillary's Scheme",
  'Why Courage Matters',
  'Dereliction of Duty',
  'The Real America',
  'Off with Their Heads',
  'Legacy',
  'Ten Minutes from Normal',
  'Hollywood Interrupted',
  'Deliver Us from Evil',
  'Let Freedom Ring',
  'Persecution',
  'Spin Sisters',
  "The O'Reilly Factor",
  '

In [37]:
df_community = pd.DataFrame
community_1 = community_list[0]
community_2 = community_list[1]
community_3 = community_list[2]
community_4 = community_list[3]

book_list = []
for i in community_1:
  book_list.append(i)
for i in community_2:
  book_list.append(i)
for i in community_3:
  book_list.append(i)
for i in community_4:
  book_list.append(i)
print(len(book_list))
community_dict = {}

for item in book_list:
        if item in community_1:
            community_dict[item] = 1
        if item in community_2:
            community_dict[item] = 2
        if item in community_3:
            community_dict[item] = 3
        if item in community_4:
            community_dict[item] = 4



print(community_dict)

64
{"Charlie Wilson's War": 1, 'Ghost Wars': 1, 'Sleeping With the Devil': 1, 'Why America Slept': 1, 'Dangerous Dimplomacy': 1, 'Bush vs. the Beltway': 1, '1000 Years for Revenge': 1, 'The Man Who Warned America': 1, "Who's Looking Out for You?": 2, 'Slander': 2, 'The Right Man': 2, 'Shut Up and Sing': 2, 'Those Who Trespass': 2, 'The French Betrayal of America': 2, 'Hating America': 2, 'Losing Bin Laden': 2, 'The Death of Right and Wrong': 2, 'The Third Terrorist': 2, "Rumsfeld's War": 2, 'Betrayal': 2, 'Bush Country': 2, 'Fighting Back': 2, 'Tales from the Left Coast': 2, 'The Enemy Within': 2, 'Breakdown': 2, 'We Will Prevail': 2, 'Power Plays': 2, 'Give Me a Break': 2, 'A National Party No More': 2, 'Useful Idiots': 2, "Hillary's Scheme": 2, 'Why Courage Matters': 2, 'Dereliction of Duty': 2, 'The Real America': 2, 'Off with Their Heads': 2, 'Legacy': 2, 'Ten Minutes from Normal': 2, 'Hollywood Interrupted': 2, 'Deliver Us from Evil': 2, 'Let Freedom Ring': 2, 'Persecution': 2, 'S

In [38]:
df['community_louvain'] = df['Label'].map(community_dict)

In [39]:
df

,Id,Label,Interval,Value,degree_centrality,betweenness_centrality,communtiy_modularity,community_louvain
0,0,1000 Years for Revenge,NaN,Neutral,0.057692,0.007433,1,1.0
1,1,Bush vs. the Beltway,NaN,Conservative,0.038462,0.000049,1,1.0
2,2,Charlie Wilson's War,NaN,Conservative,0.038462,0.000521,1,1.0
3,3,Losing Bin Laden,NaN,Conservative,0.221154,0.076093,1,2.0
4,4,Sleeping With the Devil,NaN,Neutral,0.076923,0.062928,1,1.0
...,...,...,...,...,...,...,...,...
100,100,Thieves in High Places,NaN,Liberal,0.115385,0.007119,2,NaN
101,101,Shrub,NaN,Liberal,0.048077,0.000943,2,NaN
102,102,Buck Up Suck Up,NaN,Liberal,0.038462,0.040995,2,NaN
103,103,The Future of Freedom,NaN,Neutral,0.019231,0.000000,3,4.0


In [40]:
category_counts = df['community_louvain'].value_counts()

# print the category counts
print(category_counts)

2.0    42
4.0    11
1.0     8
3.0     3
Name: community_louvain, dtype: int64


In [41]:
fig = px.histogram(df, x='community_louvain', color='Value', barmode='group',
            color_discrete_map={'Conservative': '#D25565', 'Neutral': '#F0B775', 'Liberal': '#2E94B9'})

# update the layout of the figure
fig.update_layout(title='Community Detection of Political Books on Amazon with Louvin Community Detection',
                  xaxis_title='Community',
                  yaxis_title='Count',
                  xaxis={'categoryorder': 'total descending'},
                  height=400, width=900)

# show the plot
fig.show()